In [3]:
import pandas as pd
import re

# import dataset 
dataset = pd.read_csv('../dataset/resume_dataset.csv')

# drop unecessary columns
dataset = dataset.drop(columns=['ID', 'Resume_html', 'Category'])

# set settings for printing
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 1000)

print(dataset[:10])

ImportError: attempted relative import with no known parent package

In [47]:
# check for missing values in each row
missing_values_per_row = dataset.isnull().any(axis=1)

# display rows with missing values
rows_with_missing_values = dataset[missing_values_per_row]
print(rows_with_missing_values)

Empty DataFrame
Columns: [Resume_str]
Index: []


In [48]:
segmented_sentence_dataset = []

# iterate over all resumes
for resume in dataset['Resume_str']:
   # split the text when there is two or more spaces between
   segmented_sentence_dataset += (re.split(r'\s\s+', resume))

for sentence in segmented_sentence_dataset[:10]:
   print([sentence])

['']
['HR ADMINISTRATOR/MARKETING ASSOCIATE']
['HR ADMINISTRATOR']
['Summary']
['Dedicated Customer Service Manager with 15+ years of experience in Hospitality and Customer Service Management.']
['Respected builder and leader of customer-focused teams; strives to instill a shared, enthusiastic commitment to customer service.']
['Highlights']
['Focused on customer satisfaction']
['Team management']
['Marketing savvy']


In [49]:
cleaned_sentence_dataset = []

for sentence in segmented_sentence_dataset:
    # convert item names to lowercase
    sentence = sentence.lower()

    # substitute year to [YEAR] token
    sentence = re.sub(r"(20\d\d|19\d\d)", '[YEAR]', sentence)

    # substitute numbers to [NUMBER] token
    sentence = re.sub(r"\d+\.?\d*\+?", '[NUMBER]', sentence)
    
    # remove special characters (e.g., punctuations)
    special_characters = r"[!\"#$%&()*+,/:;<=>?@^_{|}~•·]"
    sentence = re.sub(special_characters, ' ', sentence)
    new_sentence = []
    for word in sentence.split(' '):
        if (re.search(r"[\.\-]", word)):
            # handler of abbreviations that use '.' (e.g., B.S.)
            if (re.search(r"^[^\.]+\.$|^.$", word)):
                word = re.sub(r"\.", ' ', word)
            # handler of misplaced '-' and avoid removing '-' for hypenated words (e.g., Mother-in-law)
            if (re.search(r"[^\s]+-[^\s]+", word) == None):
                word = re.sub(r"-", ' ', word)
        new_sentence.append(word)
    sentence = ' '.join(new_sentence)

    # remove white spaces
    sentence = ' '.join(sentence.split())

    # explicit removal of observed noise words
    noise_words = ['n a', 'company name']
    if (sentence in noise_words):
        sentence = ''

    cleaned_sentence_dataset.append(sentence)

# for sentence in cleaned_sentence_dataset[:10]:
#    print([sentence])



In [50]:
sentence_dataset = []

# remove data with less than 2 words
for sentence in cleaned_sentence_dataset:
    if (sentence.strip() != '' 
            and len(sentence.split(' ')) > 1
            and not re.search(r"\[YEAR\]", sentence)):
        sentence_dataset.append(sentence)

for sentence in sentence_dataset[:10]:
   print([sentence])

['hr administrator marketing associate']
['hr administrator']
['dedicated customer service manager with [NUMBER] years of experience in hospitality and customer service management']
['respected builder and leader of customer-focused teams strives to instill a shared enthusiastic commitment to customer service']
['focused on customer satisfaction']
['team management']
['marketing savvy']
['conflict resolution techniques']
['training and development']
['skilled multi-tasker']
